In [110]:
val reportFolder = "reports"
val executionRounds = 5

In [111]:
import java.io.File
import java.util.Properties

val properties = Properties()
File("gradle.properties").bufferedReader().use {
    properties.load(it)
}

val executedTask = properties.getProperty("scenario.task") ?: "assemble"
val projectPath = properties.getProperty("project.path") ?: properties.getProperty("project.git.url")

val projectName: String = projectPath?.substringAfterLast("\\")?.substringAfterLast("/")?.removeSuffix(".git") ?: "User"

projectName 

cryptography-kotlin

In [112]:
enum class CompilerMetrics(val readableName: String) {
    GRADLE_TASK("Total Gradle task time"),

    COMPILATION_ROUND("Sources compilation round"),
    
    //compiler metrics
    COMPILER_INITIALIZATION("Compiler initialization time"),
    CODE_ANALYSIS("Compiler code analysis"),
    IR_TRANSLATION("Compiler IR translation"),
    IR_LOWERING("Compiler IR lowering"),
    IR_GENERATION("Compiler IR generation"),
    ;
}

In [113]:
%use dataframe

In [114]:
import java.io.File
import org.jetbrains.kotlinx.dataframe.DataFrame

//Json report print time in milliseconds 
fun readJsonReportFile(file: File): Map<CompilerMetrics, Long?>? {
    if (file.extension != "json" || file.readText().isBlank())
        return null
    val report = DataFrame.read(file)
//    if (report.get("startParameters").get("tasks") as? String != executedTask)
//        return null
    val timeMetrics = report.get("aggregatedMetrics").get("buildTimes").get("buildTimesNs")[0] as DataRow<*>

    return CompilerMetrics.values().associate {
        val value = timeMetrics.getOrNull(it.name)?.let {
            when (it) {
                is Int -> it.toLong()
                is Long -> it
                is String -> it.toLong()
                else -> throw IllegalStateException("Unknown type ${it.javaClass} to convert to Long")
            }
        }
        Pair(it, value)
    }
}

In [116]:
import kotlin.streams.toList

//Text report print time in seconds with 2 decemal digits, so there is an error during conversion to nanoSeconds
fun readTextReportFile(file: File): Map<CompilerMetrics, Long?>? {
    val isValidReportFile = file.extension == "txt" && file.readText().isNotEmpty()  && file.readText().contains("tasks = [$executedTask]")
    if (!isValidReportFile) return null
    
    val aggregatedTimeMetrics = file.bufferedReader().use {
        it.lines().dropWhile { !it.startsWith("Time metrics:") }.takeWhile { it.isNotBlank() }.toList()
    }

    return CompilerMetrics.values().associate { metric ->
        val metricLine = aggregatedTimeMetrics.firstOrNull { it.trim().startsWith(metric.readableName) }
        val value = metricLine?.substringAfter(": ")?.substringBefore(" ")?.replace(",", "")?.toDouble()
        Pair(metric, value?.times(1_000_000_000)?.toLong())
    }
}


# Read compiler metrics


In [117]:
%use kandy

In [118]:
import org.jetbrains.kotlinx.dataframe.api.toDataFrame
import java.nio.file.Files
import kotlin.io.path.Path

fun readMetrics(
    directory: File,
    readReportFile: (File) -> Map<CompilerMetrics, Long?>?
) = directory.listFiles().filter { it.name.startsWith(projectName) } //we ignore buildSrc and included builds right now
    .sortedByDescending(File::lastModified)
    .mapNotNull { readReportFile(it) }
    .take(executionRounds)
    .toList()

fun collectMeanValue(
    metrics: List<Map<CompilerMetrics, Long?>>,
): Map<String, Any?> {
    val rows = CompilerMetrics.values().map { compilerMetric ->
        val values = metrics.map { it[compilerMetric] }.filterNotNull()
        Pair(compilerMetric.name, values)
    }.filter { 
        val allValuesPresent = (it.second.size == executionRounds)
        if (!allValuesPresent) {
            println("Unable to calculate mean value for ${it.first}, only ${it.second.size} runs are found")
        }
        allValuesPresent
    }.toTypedArray()

    val meanValue = dataFrameOf(*rows).mean()

    return CompilerMetrics.values().associate {
        Pair(it.name, meanValue.getOrNull(it.name))
    }
} 


# Collect data into dataframe

In [120]:
val kotlinVersionColumn = "kotlinVersion"
val buildScenarioColumn = "buildScenario"

val columnNames = CompilerMetrics.values().map { it.name }.toMutableList()
columnNames.add(kotlinVersionColumn)
columnNames.add(buildScenarioColumn)

fun collectResultsIntoDataFrame(
    directory: File,
    readReportFile: (File) -> Map<CompilerMetrics, Long?>?,
): AnyFrame {
    val rows = directory.listFiles().flatMap { versionDir ->
        versionDir.listFiles().map { scenarioDir ->
            val buildMetrics = readMetrics(scenarioDir, readReportFile)
            //rename scenarios for graphics only
            val scenario = when (scenarioDir.name) {
                "clean_build" -> "Clean build"
                "incremental_abi_build" -> "Incremental build for ABI changes"
                "incremental_non_abi_build" -> "Incremental build for non ABI changes"
                else -> scenarioDir.name
            }

            println("\n Validate \'$scenario\' scenario for ${versionDir.name} kotlin version")
            val row = collectMeanValue(buildMetrics).toMutableMap()
            row.put(kotlinVersionColumn, versionDir.name)
            row.put(buildScenarioColumn, scenario)
            row
        }
    }

    val db = columnNames.map { compilerMetric -> Pair(compilerMetric, rows.map { it[compilerMetric] })}.toTypedArray()
    return dataFrameOf(*db)
}

In [121]:

val dataFrame = collectResultsIntoDataFrame(File(reportFolder), ::readTextReportFile)
    .sortBy(kotlinVersionColumn)
val scenarios = dataFrame[buildScenarioColumn]



 Validate 'Incremental build for ABI changes' scenario for 1.9.23 kotlin version

 Validate 'Incremental build for non ABI changes' scenario for 1.9.23 kotlin version

 Validate 'Clean build' scenario for 1.9.23 kotlin version

 Validate 'Incremental build for ABI changes' scenario for 2.0.0-RC1 kotlin version

 Validate 'Incremental build for non ABI changes' scenario for 2.0.0-RC1 kotlin version

 Validate 'Clean build' scenario for 2.0.0-RC1 kotlin version


In [122]:
val plots = scenarios.distinct().toList().map { scenario ->
    dataFrame.filter { buildScenario == scenario }.plot {
        layout.title = "$projectName project - $scenario"
        bars {
            x(kotlinVersionColumn) {
                axis.name = "Kotlin version"
            }
            y(CompilerMetrics.COMPILATION_ROUND.name) {
                axis.name = "Compiler build time"
            }
        }
        
    }
}
plotGrid(plots, 1)

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="KLsEPC"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"layout":{
"name":"grid",
"ncol":1,
"nrow":3,
"fit":true,
"align":false
},
"figures":[{
"ggtitle":{
"text":"cryptography-kotlin project - Incremental build for ABI changes"
},
"mapping":{
},
"data":{
"kotlinVersion":["1.9.23","2.0.0-RC1"],
"COMPILATION_ROUND":[3.0E8,7.86E8]
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"discrete":true,
"name":"Kotlin version"
},{
"aesthetic":"y",
"name":"Compiler build time",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"kotlinVersion",
"y":"COMPILATION_ROUND"
},
"stat":"identity",
"sampling":"none",
"position":"dodge",
"geom":"bar",
"data":{
}
}]
},{
"ggtitle":{
"text":"cryptography-kotlin project - Incremental build for non ABI changes"
},
"mapping":{
},
"data":{
"kotlinVersion":["1.9.23","2.0.0-RC1"],
"COMPILATION_ROUND":[4.16E8,7.58E8]
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"discrete":true,
"name":"Kotlin version"
},{
"aesthetic":"y",
"name":"Compiler build time",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"kotlinVersion",
"y":"COMPILATION_ROUND"
},
"stat":"identity",
"sampling":"none",
"position":"dodge",
"geom":"bar",
"data":{
}
}]
},{
"ggtitle":{
"text":"cryptography-kotlin project - Clean build"
},
"mapping":{
},
"data":{
"kotlinVersion":["1.9.23","2.0.0-RC1"],
"COMPILATION_ROUND":[2.172E9,2.724E9]
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"discrete":true,
"name":"Kotlin version"
},{
"aesthetic":"y",
"name":"Compiler build time",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"kotlinVersion",
"y":"COMPILATION_ROUND"
},
"stat":"identity",
"sampling":"none",
"position":"dodge",
"geom":"bar",
"data":{
}
}]
}],
"kind":"subplots"
};
 var plotContainer = document.getElementById("KLsEPC");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1.9.23 
 
 
 
 
 
 
 
 
 2.0.0-RC1 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 200M 
 
 
 
 
 
 
 400M 
 
 
 
 
 
 
 600M 
 
 
 
 
 
 
 800M 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 cryptography-kotlin project - Incremental build for ABI changes 
 
 
 
 
 Compiler build time 
 
 
 
 
 Kotlin version 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1.9.23 
 
 
 
 
 
 
 
 
 2.0.0-RC1 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 200M 
 
 
 
 
 
 
 400M 
 
 
 
 
 
 
 600M 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 cryptography-kotlin project - Incremental build for non ABI changes 
 
 
 
 
 Compiler build time 
 
 
 
 
 Kotlin version 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1.9.23 
 
 
 
 
 
 
 
 
 2.0.0-RC1 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 1G 
 
 
 
 
 
 
 2G 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 cryptography-kotlin project - Clean build 
 
 
 
 
 Compiler build time 
 
 
 
 
 Kotlin version

In [124]:

fun gradleBuildTime(scenarios: List<String>, name: String) =
    dataFrame.filter { buildScenario in scenarios }.plot {
        layout.title = "$projectName project - $name"
        bars {
            x(buildScenarioColumn) {
                axis.name = "Gradle build time"
            }
            y(GRADLE_TASK.name) {
                axis.name = "Time in nanoseconds"
            }
            fillColor(kotlinVersionColumn) {
                legend.name = "Kotlin version"
            }
        }
    }


In [126]:
gradleBuildTime(listOf("Clean build"), "Clean build")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="y3aXZD"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"ggtitle":{
"text":"cryptography-kotlin project - Clean build"
},
"mapping":{
},
"data":{
"GRADLE_TASK":[1.7614E10,1.75379999998E10],
"kotlinVersion":["1.9.23","2.0.0-RC1"],
"buildScenario":["Clean build","Clean build"]
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"discrete":true,
"name":"Gradle build time"
},{
"aesthetic":"y",
"name":"Time in nanoseconds",
"limits":[null,null]
},{
"aesthetic":"fill",
"discrete":true,
"name":"Kotlin version"
}],
"layers":[{
"mapping":{
"x":"buildScenario",
"y":"GRADLE_TASK",
"fill":"kotlinVersion"
},
"stat":"identity",
"sampling":"none",
"position":"dodge",
"geom":"bar",
"data":{
}
}]
};
 var plotContainer = document.getElementById("y3aXZD");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Clean build 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 5G 
 
 
 
 
 
 
 10G 
 
 
 
 
 
 
 15G 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 cryptography-kotlin project - Clean build 
 
 
 
 
 Time in nanoseconds 
 
 
 
 
 Gradle build time 
 
 
 
 
 
 
 
 
 Kotlin version 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1.9.23 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 2.0.0-RC1

In [127]:
gradleBuildTime(listOf("Incremental build for ABI changes", "Incremental build for non ABI changes"), "Incremental build")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="t0nmKk"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"ggtitle":{
"text":"cryptography-kotlin project - Incremental build"
},
"mapping":{
},
"data":{
"GRADLE_TASK":[4.0039999996E9,4.3119999996E9,6.19E9,5.872E9],
"kotlinVersion":["1.9.23","1.9.23","2.0.0-RC1","2.0.0-RC1"],
"buildScenario":["Incremental build for ABI changes","Incremental build for non ABI changes","Incremental build for ABI changes","Incremental build for non ABI changes"]
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"discrete":true,
"name":"Gradle build time"
},{
"aesthetic":"y",
"name":"Time in nanoseconds",
"limits":[null,null]
},{
"aesthetic":"fill",
"discrete":true,
"name":"Kotlin version"
}],
"layers":[{
"mapping":{
"x":"buildScenario",
"y":"GRADLE_TASK",
"fill":"kotlinVersion"
},
"stat":"identity",
"sampling":"none",
"position":"dodge",
"geom":"bar",
"data":{
}
}]
};
 var plotContainer = document.getElementById("t0nmKk");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Incremental build for ABI changes 
 
 
 
 
 
 
 
 
 Incremental build for non ABI changes 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 5G 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 cryptography-kotlin project - Incremental build 
 
 
 
 
 Time in nanoseconds 
 
 
 
 
 Gradle build time 
 
 
 
 
 
 
 
 
 Kotlin version 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1.9.23 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 2.0.0-RC1

In [128]:
fun kotlinCompilationTime(scenarios: List<String>, name: String) =
dataFrame.filter { buildScenario in scenarios }.plot {
    layout.title = "$projectName project - $name"
    bars {
        x(buildScenarioColumn) {
            axis.name = "Kotlin compilation time"
        }
        y(GRADLE_TASK.name) {
            axis.name = "Time in nanoseconds"
        }
        fillColor(kotlinVersionColumn) {
            legend.name = "Kotlin version"
        }
    }
}


In [129]:
kotlinCompilationTime(listOf("Clean build"), "Clean build")


<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="pNtQUo"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"ggtitle":{
"text":"cryptography-kotlin project - Clean build"
},
"mapping":{
},
"data":{
"GRADLE_TASK":[1.7614E10,1.75379999998E10],
"kotlinVersion":["1.9.23","2.0.0-RC1"],
"buildScenario":["Clean build","Clean build"]
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"discrete":true,
"name":"Kotlin compilation time"
},{
"aesthetic":"y",
"name":"Time in nanoseconds",
"limits":[null,null]
},{
"aesthetic":"fill",
"discrete":true,
"name":"Kotlin version"
}],
"layers":[{
"mapping":{
"x":"buildScenario",
"y":"GRADLE_TASK",
"fill":"kotlinVersion"
},
"stat":"identity",
"sampling":"none",
"position":"dodge",
"geom":"bar",
"data":{
}
}]
};
 var plotContainer = document.getElementById("pNtQUo");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Clean build 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 5G 
 
 
 
 
 
 
 10G 
 
 
 
 
 
 
 15G 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 cryptography-kotlin project - Clean build 
 
 
 
 
 Time in nanoseconds 
 
 
 
 
 Kotlin compilation time 
 
 
 
 
 
 
 
 
 Kotlin version 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1.9.23 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 2.0.0-RC1

In [130]:
kotlinCompilationTime(listOf("Incremental build for ABI changes", "Incremental build for non ABI changes"), "Incremental build")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="GOAayn"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"ggtitle":{
"text":"cryptography-kotlin project - Incremental build"
},
"mapping":{
},
"data":{
"GRADLE_TASK":[4.0039999996E9,4.3119999996E9,6.19E9,5.872E9],
"kotlinVersion":["1.9.23","1.9.23","2.0.0-RC1","2.0.0-RC1"],
"buildScenario":["Incremental build for ABI changes","Incremental build for non ABI changes","Incremental build for ABI changes","Incremental build for non ABI changes"]
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"discrete":true,
"name":"Kotlin compilation time"
},{
"aesthetic":"y",
"name":"Time in nanoseconds",
"limits":[null,null]
},{
"aesthetic":"fill",
"discrete":true,
"name":"Kotlin version"
}],
"layers":[{
"mapping":{
"x":"buildScenario",
"y":"GRADLE_TASK",
"fill":"kotlinVersion"
},
"stat":"identity",
"sampling":"none",
"position":"dodge",
"geom":"bar",
"data":{
}
}]
};
 var plotContainer = document.getElementById("GOAayn");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Incremental build for ABI changes 
 
 
 
 
 
 
 
 
 Incremental build for non ABI changes 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 5G 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 cryptography-kotlin project - Incremental build 
 
 
 
 
 Time in nanoseconds 
 
 
 
 
 Kotlin compilation time 
 
 
 
 
 
 
 
 
 Kotlin version 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1.9.23 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 2.0.0-RC1

# Compare Gradle and Compiler time

In [131]:

fun getPlotForGradleCompilerTime(scenario: String) = dataFrame.filter { it["buildScenario"] == scenario }
        .gather { GRADLE_TASK.name and COMPILATION_ROUND.name }.into("metric", "time_ns")
        .update("metric").with {
            when (it) {
                "GRADLE_TASK"  -> "Gradle time"
                "COMPILATION_ROUND" -> "Kotlin Compiler time"
                else -> throw IllegalStateException("Unknown metrics: $it")
            }
        }
        .sortBy("time_ns")
        .plot {
            layout {
                title = "$projectName project"
                subtitle = scenario
                xAxisLabel = "Gradle compilation breakdown"
                theme().legendPositionTop()
            }
            barsH {
                y(kotlinVersion) {
                    axis.name = "Kotin version"
                }
                x("time_ns"<Double>()) {
                    axis.name = "Time in nanoseconds"
                }
                fillColor("metric") {
                    legend.name = "Build time"
                }
                position = Position.identity()
            }
        }




In [132]:
val versionByScenario =
    scenarios.distinct().toList().associate { name -> Pair(name as String, getPlotForGradleCompilerTime(name)) }

In [133]:
versionByScenario.get("Clean build")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="EvGRH4"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"ggtitle":{
"text":"cryptography-kotlin project",
"subtitle":"Clean build"
},
"mapping":{
},
"data":{
"kotlinVersion":["1.9.23","2.0.0-RC1","2.0.0-RC1","1.9.23"],
"metric":["Kotlin Compiler time","Kotlin Compiler time","Gradle time","Gradle time"],
"time_ns":[2.172E9,2.724E9,1.75379999998E10,1.7614E10]
},
"kind":"plot",
"scales":[{
"aesthetic":"y",
"discrete":true,
"name":"Kotin version"
},{
"aesthetic":"x",
"name":"Time in nanoseconds",
"limits":[null,null]
},{
"aesthetic":"fill",
"discrete":true,
"name":"Build time"
},{
"aesthetic":"x",
"name":"Gradle compilation breakdown"
}],
"layers":[{
"mapping":{
"y":"kotlinVersion",
"x":"time_ns",
"fill":"metric"
},
"stat":"identity",
"orientation":"y",
"sampling":"none",
"position":"identity",
"geom":"bar",
"data":{
}
}]
};
 var plotContainer = document.getElementById("EvGRH4");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 
 
 5G 
 
 
 
 
 
 
 
 
 10G 
 
 
 
 
 
 
 
 
 15G 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1.9.23 
 
 
 
 
 
 
 2.0.0-RC1 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 cryptography-kotlin project 
 
 
 
 
 Clean build 
 
 
 
 
 Kotin version 
 
 
 
 
 Gradle compilation breakdown 
 
 
 
 
 
 
 
 
 Build time 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Kotlin Compiler time 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Gradle time

In [134]:
versionByScenario.get("Incremental build for non ABI changes")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="r4QaKo"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"ggtitle":{
"text":"cryptography-kotlin project",
"subtitle":"Incremental build for non ABI changes"
},
"mapping":{
},
"data":{
"kotlinVersion":["1.9.23","2.0.0-RC1","1.9.23","2.0.0-RC1"],
"metric":["Kotlin Compiler time","Kotlin Compiler time","Gradle time","Gradle time"],
"time_ns":[4.16E8,7.58E8,4.3119999996E9,5.872E9]
},
"kind":"plot",
"scales":[{
"aesthetic":"y",
"discrete":true,
"name":"Kotin version"
},{
"aesthetic":"x",
"name":"Time in nanoseconds",
"limits":[null,null]
},{
"aesthetic":"fill",
"discrete":true,
"name":"Build time"
},{
"aesthetic":"x",
"name":"Gradle compilation breakdown"
}],
"layers":[{
"mapping":{
"y":"kotlinVersion",
"x":"time_ns",
"fill":"metric"
},
"stat":"identity",
"orientation":"y",
"sampling":"none",
"position":"identity",
"geom":"bar",
"data":{
}
}]
};
 var plotContainer = document.getElementById("r4QaKo");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 
 
 2G 
 
 
 
 
 
 
 
 
 4G 
 
 
 
 
 
 
 
 
 6G 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1.9.23 
 
 
 
 
 
 
 2.0.0-RC1 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 cryptography-kotlin project 
 
 
 
 
 Incremental build for non ABI changes 
 
 
 
 
 Kotin version 
 
 
 
 
 Gradle compilation breakdown 
 
 
 
 
 
 
 
 
 Build time 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Kotlin Compiler time 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Gradle time

In [135]:
versionByScenario.get("Incremental build for ABI changes")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="FnWwqO"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"ggtitle":{
"text":"cryptography-kotlin project",
"subtitle":"Incremental build for ABI changes"
},
"mapping":{
},
"data":{
"kotlinVersion":["1.9.23","2.0.0-RC1","1.9.23","2.0.0-RC1"],
"metric":["Kotlin Compiler time","Kotlin Compiler time","Gradle time","Gradle time"],
"time_ns":[3.0E8,7.86E8,4.0039999996E9,6.19E9]
},
"kind":"plot",
"scales":[{
"aesthetic":"y",
"discrete":true,
"name":"Kotin version"
},{
"aesthetic":"x",
"name":"Time in nanoseconds",
"limits":[null,null]
},{
"aesthetic":"fill",
"discrete":true,
"name":"Build time"
},{
"aesthetic":"x",
"name":"Gradle compilation breakdown"
}],
"layers":[{
"mapping":{
"y":"kotlinVersion",
"x":"time_ns",
"fill":"metric"
},
"stat":"identity",
"orientation":"y",
"sampling":"none",
"position":"identity",
"geom":"bar",
"data":{
}
}]
};
 var plotContainer = document.getElementById("FnWwqO");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 
 
 2G 
 
 
 
 
 
 
 
 
 4G 
 
 
 
 
 
 
 
 
 6G 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1.9.23 
 
 
 
 
 
 
 2.0.0-RC1 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 cryptography-kotlin project 
 
 
 
 
 Incremental build for ABI changes 
 
 
 
 
 Kotin version 
 
 
 
 
 Gradle compilation breakdown 
 
 
 
 
 
 
 
 
 Build time 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Kotlin Compiler time 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Gradle time

# Collect Compiler metrics

In [136]:
import org.jetbrains.kotlinx.kandy.ir.scale.PositionalContinuousScale
import org.jetbrains.kotlinx.kandy.ir.scale.PositionalScale

val metricsList = dataFrame
    .gather { COMPILER_INITIALIZATION.name and CODE_ANALYSIS.name and IR_TRANSLATION.name and IR_LOWERING.name and IR_GENERATION.name }.into("metric", "time_ns")


fun getPlotForCompilerMetrics(scenario: String) = metricsList.filter { it["buildScenario"] == scenario }
    .plot {
    layout {
        title = "$projectName project"
        subtitle = scenario
        xAxisLabel = "Kotlin compilation breakdown"
        style { 
            legend {
                position = LegendPosition.Top
            }
        }
        size = Pair(1000, 300)
    }
    barsH {
        y(kotlinVersion) {
            axis.name = "Kotlin version"
        }
        x("time_ns"<Double>()) {
            axis.name = "Time in nanoseconds"
        }
        fillColor("metric")
        position = Position.stack()
        
    }
}


In [137]:
val compilerMetrics = scenarios.distinct().toList().associate { Pair(it as String, getPlotForCompilerMetrics(it)) }

In [138]:
compilerMetrics.get("Clean build") 


<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="CgKRpk"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"ggtitle":{
"text":"cryptography-kotlin project",
"subtitle":"Clean build"
},
"mapping":{
},
"data":{
"kotlinVersion":["1.9.23","1.9.23","1.9.23","1.9.23","1.9.23","2.0.0-RC1","2.0.0-RC1","2.0.0-RC1","2.0.0-RC1","2.0.0-RC1"],
"metric":["COMPILER_INITIALIZATION","CODE_ANALYSIS","IR_TRANSLATION","IR_LOWERING","IR_GENERATION","COMPILER_INITIALIZATION","CODE_ANALYSIS","IR_TRANSLATION","IR_LOWERING","IR_GENERATION"],
"time_ns":[1.2E8,3.14E8,8.0E7,8.6E7,1.2E8,1.46E8,3.72E8,4.08E8,2.12E8,1.34E8]
},
"ggsize":{
"width":1000.0,
"height":300.0
},
"kind":"plot",
"scales":[{
"aesthetic":"y",
"discrete":true,
"name":"Kotlin version"
},{
"aesthetic":"x",
"name":"Time in nanoseconds",
"limits":[null,null]
},{
"aesthetic":"fill",
"discrete":true
},{
"aesthetic":"x",
"name":"Kotlin compilation breakdown"
}],
"layers":[{
"mapping":{
"y":"kotlinVersion",
"x":"time_ns",
"fill":"metric"
},
"stat":"identity",
"orientation":"y",
"sampling":"none",
"position":"stack",
"geom":"bar",
"data":{
}
}],
"theme":{
"axis_ontop":false,
"axis_ontop_y":false,
"axis_ontop_x":false,
"legend_position":"top"
}
};
 var plotContainer = document.getElementById("CgKRpk");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 
 
 
 
 
 
 100M 
 
 
 
 
 
 
 
 
 200M 
 
 
 
 
 
 
 
 
 300M 
 
 
 
 
 
 
 
 
 400M 
 
 
 
 
 
 
 
 
 500M 
 
 
 
 
 
 
 
 
 600M 
 
 
 
 
 
 
 
 
 700M 
 
 
 
 
 
 
 
 
 800M 
 
 
 
 
 
 
 
 
 900M 
 
 
 
 
 
 
 
 
 1.0G 
 
 
 
 
 
 
 
 
 1.1G 
 
 
 
 
 
 
 
 
 1.2G 
 
 
 
 
 
 
 
 
 1.3G 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1.9.23 
 
 
 
 
 
 
 2.0.0-RC1 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 cryptography-kotlin project 
 
 
 
 
 Clean build 
 
 
 
 
 Kotlin version 
 
 
 
 
 Kotlin compilation breakdown 
 
 
 
 
 
 
 
 
 metric 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 COMPILER_INITIALIZATION 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 CODE_ANALYSIS 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 IR_TRANSLATION 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 IR_LOWERING 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 IR_GENERATION

In [139]:
compilerMetrics.get("Incremental build for non ABI changes") 

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="wQPYGw"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"ggtitle":{
"text":"cryptography-kotlin project",
"subtitle":"Incremental build for non ABI changes"
},
"mapping":{
},
"data":{
"kotlinVersion":["1.9.23","1.9.23","1.9.23","1.9.23","1.9.23","2.0.0-RC1","2.0.0-RC1","2.0.0-RC1","2.0.0-RC1","2.0.0-RC1"],
"metric":["COMPILER_INITIALIZATION","CODE_ANALYSIS","IR_TRANSLATION","IR_LOWERING","IR_GENERATION","COMPILER_INITIALIZATION","CODE_ANALYSIS","IR_TRANSLATION","IR_LOWERING","IR_GENERATION"],
"time_ns":[4.8E7,5.8E7,1.0E7,1.0E7,4000000.0,2.0E7,1.18E8,9.6E7,5.4E7,6.0E7]
},
"ggsize":{
"width":1000.0,
"height":300.0
},
"kind":"plot",
"scales":[{
"aesthetic":"y",
"discrete":true,
"name":"Kotlin version"
},{
"aesthetic":"x",
"name":"Time in nanoseconds",
"limits":[null,null]
},{
"aesthetic":"fill",
"discrete":true
},{
"aesthetic":"x",
"name":"Kotlin compilation breakdown"
}],
"layers":[{
"mapping":{
"y":"kotlinVersion",
"x":"time_ns",
"fill":"metric"
},
"stat":"identity",
"orientation":"y",
"sampling":"none",
"position":"stack",
"geom":"bar",
"data":{
}
}],
"theme":{
"axis_ontop":false,
"axis_ontop_y":false,
"axis_ontop_x":false,
"legend_position":"top"
}
};
 var plotContainer = document.getElementById("wQPYGw");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 
 
 50M 
 
 
 
 
 
 
 
 
 100M 
 
 
 
 
 
 
 
 
 150M 
 
 
 
 
 
 
 
 
 200M 
 
 
 
 
 
 
 
 
 250M 
 
 
 
 
 
 
 
 
 300M 
 
 
 
 
 
 
 
 
 350M 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1.9.23 
 
 
 
 
 
 
 2.0.0-RC1 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 cryptography-kotlin project 
 
 
 
 
 Incremental build for non ABI changes 
 
 
 
 
 Kotlin version 
 
 
 
 
 Kotlin compilation breakdown 
 
 
 
 
 
 
 
 
 metric 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 COMPILER_INITIALIZATION 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 CODE_ANALYSIS 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 IR_TRANSLATION 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 IR_LOWERING 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 IR_GENERATION

In [140]:
compilerMetrics.get("Incremental build for ABI changes") 

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="Ue1v70"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"ggtitle":{
"text":"cryptography-kotlin project",
"subtitle":"Incremental build for ABI changes"
},
"mapping":{
},
"data":{
"kotlinVersion":["1.9.23","1.9.23","1.9.23","1.9.23","1.9.23","2.0.0-RC1","2.0.0-RC1","2.0.0-RC1","2.0.0-RC1","2.0.0-RC1"],
"metric":["COMPILER_INITIALIZATION","CODE_ANALYSIS","IR_TRANSLATION","IR_LOWERING","IR_GENERATION","COMPILER_INITIALIZATION","CODE_ANALYSIS","IR_TRANSLATION","IR_LOWERING","IR_GENERATION"],
"time_ns":[3.4E7,3.8E7,1.4E7,1.0E7,4000000.0,2.2E7,1.32E8,7.4E7,5.8E7,6.8E7]
},
"ggsize":{
"width":1000.0,
"height":300.0
},
"kind":"plot",
"scales":[{
"aesthetic":"y",
"discrete":true,
"name":"Kotlin version"
},{
"aesthetic":"x",
"name":"Time in nanoseconds",
"limits":[null,null]
},{
"aesthetic":"fill",
"discrete":true
},{
"aesthetic":"x",
"name":"Kotlin compilation breakdown"
}],
"layers":[{
"mapping":{
"y":"kotlinVersion",
"x":"time_ns",
"fill":"metric"
},
"stat":"identity",
"orientation":"y",
"sampling":"none",
"position":"stack",
"geom":"bar",
"data":{
}
}],
"theme":{
"axis_ontop":false,
"axis_ontop_y":false,
"axis_ontop_x":false,
"legend_position":"top"
}
};
 var plotContainer = document.getElementById("Ue1v70");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 
 
 50M 
 
 
 
 
 
 
 
 
 100M 
 
 
 
 
 
 
 
 
 150M 
 
 
 
 
 
 
 
 
 200M 
 
 
 
 
 
 
 
 
 250M 
 
 
 
 
 
 
 
 
 300M 
 
 
 
 
 
 
 
 
 350M 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1.9.23 
 
 
 
 
 
 
 2.0.0-RC1 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 cryptography-kotlin project 
 
 
 
 
 Incremental build for ABI changes 
 
 
 
 
 Kotlin version 
 
 
 
 
 Kotlin compilation breakdown 
 
 
 
 
 
 
 
 
 metric 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 COMPILER_INITIALIZATION 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 CODE_ANALYSIS 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 IR_TRANSLATION 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 IR_LOWERING 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 IR_GENERATION